**Fetch Data from MySQL Database**

Further instructions can be found here: https://www.freecodecamp.org/news/connect-python-with-sql/

In [ ]:
# Use this line if you have not installed mysql.connector
# pip install mysql-connector-python

In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [2]:
# Connect to server
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [3]:
pw = "HaNoi687302@@" # IMPORTANT! Put your MySQL Terminal password here.
db = "OAG_v1" # This is the name of the database

connection = create_server_connection("localhost", "root", pw)

MySQL Database connection successful


In [4]:
# Connect to database
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [5]:
# Add or modify databse
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [6]:
# Retrieve data
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

**NOTE:**

Change the info in **bold** in the follwing cell to retrieve data: 
1. WHERE (time_series >= **2011** AND time_series <= **2013**)
    * For time range
    * Available for year 2011 to 2020


2. AND LEFT(segment, 3) IN (SELECT airports.iata FROM airports WHERE airports.**region** = '**AF1**') 
    * For airport 1 (origin) in segment
    * **region** can be changed to **city_name OR country_name** 


3. AND RIGHT(segment, 3) IN (SELECT airports.iata FROM airports WHERE airports.**region** = '**AF2**')
    * For airport 2 (destination) in segment
    * **region** can be changed to **city_name OR country_name**

If seeing "Error 1055(42000): Expression #2 of SELECT list is not in GROUP BY clause and contains...", fix with this: 

**SET GLOBAL sql_mode=(SELECT REPLACE(@@sql_mode,'ONLY_FULL_GROUP_BY',''));**

In [16]:
# Define query (in SQL)
# Change the following to 

q1 = """
SELECT segment, time_series, 
SUM(total_est_pax), SUM(cabin_first), SUM(cabin_business), 
SUM(cabin_premium_economy), SUM(cabin_full_y), SUM(cabin_discount_economy), 
AVG(avgfare_total), AVG(avgfare_first), AVG(avgfare_business), 
AVG(avgfare_premium_economy), AVG(avgfare_full_y), AVG(avgfare_discount_economy)

FROM segment_traffic

WHERE (time_series >= 2011 AND time_series <= 2013) 
AND LEFT(segment, 3) IN (SELECT airports.iata FROM airports WHERE airports.region = 'AF1') 
AND RIGHT(segment, 3) IN (SELECT airports.iata FROM airports WHERE airports.region = 'AF2')

GROUP BY time_series, segment
ORDER BY time_series ASC;
"""

# q1 = """
# SELECT route_id, dep_airport_code, arr_airport_code, time_series FROM OD_traffic 
# WHERE (dep_airport_code = 'DEN' AND arr_airport_code = 'LAX') 
# AND time_series <= 2013;
# """
connection = create_db_connection("localhost", "root", pw, db)
results = read_query(connection, q1)

MySQL Database connection successful


In [17]:
# Returns a list of lists and then creates a pandas DataFrame
# Change columns if needed

from_db = []

for result in results:
  result = list(result)
  from_db.append(result)


columns = ['segment', 'time_series', 'total_est_pax', 'cabin_first', 'cabin_business', 'cabin_premium_economy',
          'cabin_full_y', 'cabin_discount_economy', 'avgfare_total', 'avgfare_first', 'avgfare_business', 
          'avgfare_premium_economy', 'acgfare_full_y', 'avgfare_discount_economy']
df = pd.DataFrame(from_db, columns=columns)

display(df)

,segment,time_series,total_est_pax,cabin_first,cabin_business,cabin_premium_economy,cabin_full_y,cabin_discount_economy,avgfare_total,avgfare_first,avgfare_business,avgfare_premium_economy,acgfare_full_y,avgfare_discount_economy
0,CAI-JNB,2011,44289,4,4306,3465,2075,34432,345.3171,132.4472,504.9268,101.9512,258.7398,276.5041
1,CAI-LUN,2011,444,0,136,11,0,294,146.8333,0.0000,97.2083,8.0833,0.0000,50.1667
2,CMN-LAD,2011,86,0,0,0,0,86,24.8000,0.0000,0.0000,0.0000,0.0000,24.8000
3,HRG-LAD,2011,199,0,14,25,1,159,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,TIP-JNB,2011,11317,503,239,907,62,9604,88.0645,150.1613,131.0968,60.9677,29.2903,76.7419
5,CAI-JNB,2012,50886,1,5640,5700,2509,37038,324.2920,85.7876,593.6018,161.2301,226.1504,252.6018
6,CMN-LAD,2012,2091,1,27,1,4,2053,167.4444,0.0000,97.8333,0.0000,83.2778,151.3333
7,HRG-LAD,2012,94,1,6,15,1,72,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,CAI-HRE,2013,6285,0,228,973,187,4893,259.6600,0.0000,260.4400,133.0600,91.0200,158.1800
9,CAI-JNB,2013,52003,2,4963,7264,2078,37702,278.2174,0.0000,437.2174,115.5043,194.9565,168.9652


**Further Preprocess**